In [18]:
import subprocess
import time
import os 
import sys 
import io

import warnings 
warnings.filterwarnings('ignore')


import pandas as pd 
import numpy as np 



In [35]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VECM
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.api import acf, graphics, pacf
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [28]:


df = pd.read_csv('../data/attention_time_series_v2.csv')

df['issue_tweets'] = df.issue_tweets_male + df.issue_tweets_female
df['total_tweets'] = df.total_tweets_male + df.total_tweets_female
df['attention'] = df.issue_tweets / df.total_tweets


df = df.loc[(df.mip_male != 0)&(df.mip_female != 0)]



us = df[df['country'] == 'us']
uk = df[df['country'] == 'uk']

us=us.loc[us.issue != 'international_affairs']

def lag_dv(df, col, lag):
    x = df.copy()
    x = x.sort_values(by=['date','issue'])
    x[f'{col}_{lag}'] = x.groupby(['issue'])[col].shift(-lag)
    return x 

cols = ['mip_female', 'mip_male']


for col in cols:
    us = lag_dv(us, col, 1)
    us = lag_dv(us, col, 2)
    uk = lag_dv(uk, col, 1)
    uk = lag_dv(uk, col, 2)

In [29]:
us

,date,issue,issue_tweets_female,issue_tweets_male,total_tweets_female,total_tweets_male,mip_female,mip_male,country,male_attention,female_attention,attention,mip,total_tweets,date_int,issue_tweets,mip_female_1,mip_female_2,mip_male_1,mip_male_2
0,2018-01-02,crime,0.0,5.0,21.0,196.0,0.03,0.04,us,0.025510,0.000000,0.023041,0.035,217.0,0,5.0,0.03,0.03,0.02,0.02
1,2018-01-02,defense,4.0,8.0,21.0,196.0,0.09,0.07,us,0.040816,0.190476,0.055300,0.080,217.0,0,12.0,0.09,0.08,0.08,0.08
2,2018-01-02,economy,3.0,18.0,21.0,196.0,0.13,0.20,us,0.091837,0.142857,0.096774,0.165,217.0,0,21.0,0.13,0.16,0.21,0.18
3,2018-01-02,education,1.0,8.0,21.0,196.0,0.05,0.04,us,0.040816,0.047619,0.041475,0.045,217.0,0,9.0,0.06,0.06,0.06,0.06
4,2018-01-02,environment,1.0,6.0,21.0,196.0,0.08,0.07,us,0.030612,0.047619,0.032258,0.075,217.0,0,7.0,0.08,0.08,0.09,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,2021-12-21,education,18.0,31.0,273.0,667.0,0.03,0.03,us,0.046477,0.065934,0.052128,0.030,940.0,352,49.0,NaN,NaN,NaN,NaN
1831,2021-12-21,environment,17.0,31.0,273.0,667.0,0.18,0.16,us,0.046477,0.062271,0.051064,0.170,940.0,352,48.0,NaN,NaN,NaN,NaN
1832,2021-12-21,health,62.0,121.0,273.0,667.0,0.18,0.12,us,0.181409,0.227106,0.194681,0.150,940.0,352,183.0,NaN,NaN,NaN,NaN
1833,2021-12-21,immigration,4.0,22.0,273.0,667.0,0.05,0.10,us,0.032984,0.014652,0.027660,0.075,940.0,352,26.0,NaN,NaN,NaN,NaN


In [31]:
us_ts = us.set_index('date')
uk_ts = uk.set_index('date')




us_ts = us_ts[['male_attention', 'female_attention', 'mip_female_1', 'mip_male']]
uk_ts = uk_ts[['male_attention', 'female_attention', 'mip_female', 'mip_male']]




       
uk_ts = uk_ts.replace(0, np.nan).dropna()
us_ts = us_ts.replace(0, np.nan).dropna()


## get log ratio of proportional data (composition data)
uk_ts = np.log(uk_ts.div(uk_ts.sum(axis=1), axis=0)).dropna()
us_ts = np.log(us_ts.div(us_ts.sum(axis=1), axis=0)).dropna()


#uk_ts = uk_ts.diff().dropna()
#us_ts = us_ts.diff().dropna()



us_model = VAR(us_ts)
uk_model = VAR(uk_ts)

print('US: ' + us_model.select_order(10).summary().as_text())
print('UK: ' + uk_model.select_order(10).summary().as_text())

US:  VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       -8.541      -8.526   0.0001953      -8.535
1       -8.605      -8.532   0.0001832      -8.578
2       -8.669      -8.536   0.0001718      -8.619
3       -8.722      -8.531   0.0001630      -8.651
4       -8.761      -8.511   0.0001567      -8.668
5       -8.792      -8.482   0.0001520      -8.676
6       -8.805      -8.437   0.0001499      -8.668
7       -9.097     -8.669*   0.0001121      -8.937
8      -9.153*      -8.667  0.0001059*     -8.971*
9       -9.139      -8.594   0.0001074      -8.935
10      -9.139      -8.535   0.0001074      -8.913
--------------------------------------------------
UK:  VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       -10.12      -10.10   4.040e-05      -10.11
1       -10.23      -10

/Users/z.dickson/Library/Python/3.11/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/z.dickson/Library/Python/3.11/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [32]:
print('UK VIF: ' + str(variance_inflation_factor(add_constant(uk_ts).values, 1)))
print('US VIF: ' + str(variance_inflation_factor(add_constant(us_ts).values, 1)))

NameError: name 'variance_inflation_factor' is not defined